<a href="https://colab.research.google.com/github/Muskank56/RecoPhysix/blob/main/RecoPhysix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/recophysix_dataset





preprocessed_questions.csv  RecoPhysix_Dataset.csv  subjects-questions.csv.zip


In [3]:
import pandas as pd
import zipfile
import os

# Correct path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/recophysix_dataset/subjects-questions.csv.zip'
extracted_dir = 'extracted_data'
csv_file_name = 'subjects-questions.csv'
csv_file_path = os.path.join(extracted_dir, csv_file_name)

# Create a directory to extract the contents
os.makedirs(extracted_dir, exist_ok=True)

# Extract the CSV file from the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extract(csv_file_name, extracted_dir)

# Load the dataset
subject_df = pd.read_csv(csv_file_path)
display(subject_df.head())


,eng,Subject
0,An anti-forest measure is\nA. Afforestation\nB...,Biology
1,"Among the following organic acids, the acid pr...",Chemistry
2,If the area of two similar triangles are equal...,Maths
3,"In recent year, there has been a growing\nconc...",Biology
4,Which of the following statement\nregarding tr...,Physics


In [4]:
subject_count = subject_df['Subject'].value_counts()
print(subject_count)

Subject
Physics      38438
Chemistry    37767
Maths        33190
Biology      13124
Name: count, dtype: int64


In [5]:
#data preprocessing
%pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 11.3 MB/s eta 0:00:00


In [6]:
import contractions

#Expanding contractions such as I'll to I will etc.
def expand_contractions(s):
    return contractions.fix(s)

In [7]:
#reducing the size of dataset and extracting physics questions
df_physics = subject_df[subject_df['Subject']=='Physics'].sample(n = 1500)
balanced_df = pd.concat([df_physics])
balanced_df = balanced_df.sample(frac = 1)
balanced_df.reset_index(drop=True, inplace=True)
subject_df_clean = balanced_df.copy()
subject_df_clean.rename(columns = {'eng':'Questions'}, inplace = True)
print(subject_df_clean)


                                              Questions  Subject
0     Match the terms in column I with those\nof col...  Physics
1     A metal cylinder of length \( L \) is\nsubject...  Physics
2     Fill in the blank.\nThe reflection of sunlight...  Physics
3     The unified atomic mass unit is the\nstandard ...  Physics
4     State whether true or false.\nThe image formed...  Physics
...                                                 ...      ...
1495  Which of the following conclusion can be drawn...  Physics
1496  A particle is going in a spiral path as\nshown...  Physics
1497  A carbon resistor is shown in the figure.\nUsi...  Physics
1498  Drive an expression for Electric field due to ...  Physics
1499  If the angle of dip at a certain place is\n\( ...  Physics

[1500 rows x 2 columns]


In [134]:
keyword_topic_map = {
    # Easy Topics
    "Kinematics": ["motion", "velocity", "displacement", "acceleration", "graph", "uniform", "non-uniform", "rest"],
    "Optics": ["mirror", "lens", "refraction", "reflection", "focal", "image", "optical", "light"],

    #  Medium Topics
    "Mechanics": ["force", "friction", "newton", "mass", "energy", "momentum", "projectile", "work", "power", "torque"],
    "Thermodynamics": ["temperature", "heat", "gas", "entropy", "internal energy", "expansion", "laws of thermodynamics"],

    # Hard Topics
    "Electricity": ["current", "voltage", "resistance", "ohm", "circuit", "capacitor", "inductor", "charge", "battery"],
    "Modern Physics": ["quantum", "photoelectric", "relativity", "nuclear", "radioactivity", "atom", "isotope", "electron", "de broglie", "dual nature", "planck", "wave-particle", "bohr","x-rays", "gamma", "alpha", "beta", "fission", "fusion", "mass defect", "binding energy"
    ]

}

def classify_topic(question):
    question = question.lower()
    for topic, keywords in keyword_topic_map.items():
        for kw in keywords:
            if kw in question:
                return topic
    return "Mixed Questions "

subject_df_clean['topic'] = subject_df_clean['Questions'].apply(classify_topic)

#defining difficulty level according to topics
difficulty_map = {
    "Kinematics": "Easy",
    "Optics": "Easy",
    "Mechanics": "Medium",
    "Thermodynamics": "Medium",
    "Electricity": "Hard",
    "Modern Physics": "Hard",
    "Mixed Questions": "very hard"
}

subject_df_clean['Topic_Difficulty'] = subject_df_clean['topic'].map(difficulty_map)
print(subject_df_clean['topic'].value_counts())


topic
Mixed Questions     353
Kinematics          350
Mechanics           310
Electricity         192
Optics              171
Thermodynamics       84
Modern Physics       40
Name: count, dtype: int64


In [118]:
import random

# Simulating student data for Physics quiz system
subject_df_clean['student_id'] = [random.randint(1001, 1100) for _ in range(len(subject_df_clean))]
subject_df_clean['quiz_attempt'] = [random.randint(1, 10) for _ in range(len(subject_df_clean))]
subject_df_clean['score'] = [random.randint(20, 100) for _ in range(len(subject_df_clean))]
subject_df_clean['time_taken'] = [random.randint(30, 300) for _ in range(len(subject_df_clean))]

# Difficulty level based on score
subject_df_clean['difficulty'] = subject_df_clean['score'].apply(
    lambda s: 'Easy' if s < 40 else ('Medium' if s < 70 else 'Hard')
)

# Rule-based recommendation of next topic
def recommend_next(row):
    if row['score'] < 40:
        return random.choice(['kinematics','optics']).lower()
    elif row['score'] < 60:
        return random.choice(['mechanics', 'thermodynamics ']).lower()
    elif row['score'] < 80:
        return random.choice(['Electricity','modern physics']).lower()
    else:
        return 'mixed questions'.lower()


subject_df_clean['recommended_next_topic'] = subject_df_clean.apply(recommend_next, axis=1)

In [121]:
import pandas as pd
import re


# Function to extract question and options from the text
def extract_question_data(entry):
    if pd.isnull(entry):
        return pd.Series([None] * 6)

    # Extract question part
    question_match = re.split(r'\n[A-Da-d][\.\)]', entry, maxsplit=1)
    question = question_match[0].strip() if question_match else None

    # Extract options using regex
    options = re.findall(r'([A-Da-d])[\.\)]\s*(.*?)(?=\n[A-Da-d][\.\)]|\Z)', entry, re.DOTALL)

    # Must have exactly 4 options
    if len(options) != 4:
        return pd.Series([None] * 6)

    # Extract option texts only
    extracted_options = [opt[1].strip() for opt in options]

    # Check if any option is empty
    if any(opt == '' or pd.isnull(opt) for opt in extracted_options):
        return pd.Series([None] * 6)

    return pd.Series([
        question,
        extracted_options[0],
        extracted_options[1],
        extracted_options[2],
        extracted_options[3],
        None  # Placeholder for correct answer
    ])

# Apply extraction
subject_df_clean[['question', 'option_1', 'option_2', 'option_3', 'option_4', 'correct_answer']] = subject_df_clean['Questions'].apply(extract_question_data)

# Drop rows where any of the options or question is missing
df_filtered = subject_df_clean.dropna(subset=['question', 'option_1', 'option_2', 'option_3', 'option_4'])

# Optional: Group by topic
grouped = df_filtered.groupby('topic')

# Save cleaned and filtered data
df_filtered.to_csv('structured_physics_quiz.csv', index=False)

# Display result
print(df_filtered[['question', 'option_1', 'option_2', 'option_3', 'option_4', 'correct_answer', 'topic']].head())


                                            question  \
0  Match the terms in column I with those\nof col...   
5  In the series \( L C R \) circuit as shown in\...   
6  A converging lens of focal length \( f_{1} \) ...   
7  Assertion: Specific heat of a body is\nalways ...   
8        Which of the following statements is\ntrue?   

                                            option_1  \
0  Chemical effect } \\ \text { ii. Relay } & \te...   
5                    \( V=100 \) volt, \( I=2 \) amp   
6  The diameter of incident and emergent beam is ...   
7  Both assertion and reason are correct and reas...   
8  The image formed by a pinhole camera is invert...   

                                            option_2  \
0                             i-B, ii-A, iii-C, iv-D   
5                    \( V=100 \) volt, \( I=5 \) amp   
6                  The distance d can be \( f_{1} \)   
7  Both assertion and reason are correct, but rea...   
8  Light does not change its direction on refl

In [122]:
print(df_filtered['topic'].value_counts())

topic
Mixed Questions     105
Kinematics          103
Mechanics            98
Optics               62
Electricity          50
Thermodynamics       35
Modern Physics       18
Name: count, dtype: int64


In [124]:
# testing the model
import pandas as pd
sample_input = [[2, 1, 65, 3, 120]]  # topic_enc, diff_enc, score, quiz_attempt, time_taken - Note: topic_enc is not used for prediction
input_df = pd.DataFrame(sample_input, columns=['topic_enc', 'diff_enc', 'score', 'quiz_attempt', 'time_taken'])
# Select only the features used during training for prediction
predicted_topic_enc = model.predict(input_df[['diff_enc', 'score', 'quiz_attempt', 'time_taken']])[0]
predicted_topic = le_next.inverse_transform([predicted_topic_enc])[0]
print("Predicted Recommended Topic:", predicted_topic)

Predicted Recommended Topic: kinematics


In [125]:
# AI Engine - Clean & Train Model

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Clean the Data
# Drop rows with any missing required values
df_filtered = subject_df_clean.dropna(subset=[
    'question', 'option_1', 'option_2', 'option_3', 'option_4',
    'topic', 'difficulty', 'recommended_next_topic',
    'score', 'quiz_attempt', 'time_taken'
]).copy()

# Step 2: Label Encoding
le_topic = LabelEncoder()
le_diff = LabelEncoder()
le_next = LabelEncoder()

df_filtered['topic_enc'] = le_topic.fit_transform(df_filtered['topic'])
df_filtered['diff_enc'] = le_diff.fit_transform(df_filtered['difficulty'])
df_filtered['target_enc'] = le_next.fit_transform(df_filtered['recommended_next_topic'])

# Step 3: Define Features & Target
X = df_filtered[[ 'diff_enc', 'score', 'quiz_attempt', 'time_taken']]
y = df_filtered['target_enc']

# Step 4: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train Model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 6: Evaluate Model
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

# Safe inverse_transform for only used labels
unique_labels = np.unique(np.concatenate((y_test, y_pred)))
target_names_subset = le_next.inverse_transform(unique_labels)

print("\nClassification Report:\n", classification_report(
    y_test, y_pred,
    labels=unique_labels,
    target_names=target_names_subset
))


Accuracy: 0.631578947368421

Classification Report:
                  precision    recall  f1-score   support

    electricity       0.55      0.60      0.57        10
     kinematics       0.60      0.71      0.65        17
      mechanics       0.45      0.42      0.43        12
mixed questions       1.00      1.00      1.00        22
 modern physics       0.50      0.44      0.47         9
         optics       0.38      0.27      0.32        11
thermodynamics        0.53      0.57      0.55        14

       accuracy                           0.63        95
      macro avg       0.57      0.57      0.57        95
   weighted avg       0.62      0.63      0.63        95



In [126]:
import pandas as pd
all_topics = df_filtered['topic'].unique()
all_topics_enc = le_topic.transform(all_topics)
difficulty_levels = ['Easy', 'Medium', 'Hard']
difficulty_enc = le_diff.transform(difficulty_levels)
test_data = []
for topic_enc, topic_name in zip(all_topics_enc, all_topics):
    for diff_name, diff_enc_val in zip(difficulty_levels, difficulty_enc):
        test_data.append({
            'topic': topic_name,
            'topic_enc': topic_enc,
            'diff_enc': diff_enc_val,
            'score': 60 if diff_name == 'Medium' else (35 if diff_name == 'Easy' else 85),
            'quiz_attempt': 2,
            'time_taken': 100
        })
test_df = pd.DataFrame(test_data)
X_test = test_df[[ 'diff_enc', 'score', 'quiz_attempt', 'time_taken']]
test_df['predicted_topic_enc'] = model.predict(X_test)
test_df['predicted_recommendation'] = le_next.inverse_transform(test_df['predicted_topic_enc'])
result_df = test_df[['topic', 'score', 'diff_enc', 'predicted_recommendation']]
print(result_df.to_string(index=False))


           topic  score  diff_enc predicted_recommendation
  Thermodynamics     35         0               kinematics
  Thermodynamics     60         2           modern physics
  Thermodynamics     85         1          mixed questions
     Electricity     35         0               kinematics
     Electricity     60         2           modern physics
     Electricity     85         1          mixed questions
          Optics     35         0               kinematics
          Optics     60         2           modern physics
          Optics     85         1          mixed questions
       Mechanics     35         0               kinematics
       Mechanics     60         2           modern physics
       Mechanics     85         1          mixed questions
      Kinematics     35         0               kinematics
      Kinematics     60         2           modern physics
      Kinematics     85         1          mixed questions
Mixed Questions      35         0               kinemati

In [127]:
# creating quizes per topic
df_filtered['quiz_id'] = ['QZ' + str(i).zfill(4) for i in range(len(df_filtered))]
df_filtered['quiz_question'] = df_filtered['Questions']

quiz_bank = {
    topic: group[['quiz_id', 'quiz_question']].sample(min(10, len(group)))
    for topic, group in df_filtered.groupby('topic')
}

In [129]:
def recommend_next_quiz_set(row):
    current_topic = row['topic']
    current_diff = row['difficulty']
    score = row['score']

    # Decide next difficulty level
    if score < 40:
        next_diff = 'easy'
    elif score <= 60:
        next_diff = current_diff
    else:
        next_diff = 'hard'

    # Get topics at the next difficulty (excluding current topic)
    possible_topics = df_filtered[
        (df_filtered['difficulty'] == next_diff) &
        (df_filtered['topic'] != current_topic)
    ]['topic'].unique()

    # If none found (edge case), fallback to current topic
    if len(possible_topics) > 0:
        predicted_topic = np.random.choice(possible_topics)
    else:
        predicted_topic = current_topic

    # Message logic
    if score < 40:
        message = f"😐 You need more practice. Let's go back to an easier topic: {predicted_topic}."
    elif score <= 60:
        message = f"🙂 You're doing okay. Let's try another medium topic: {predicted_topic}."
    elif score <= 80:
        message = f"🔥 Great job! You're ready for a harder topic: {predicted_topic}."
    else:
        message = f"🎉 Well done! You're ready for a mixed tquestions practice : {predicted_topic}."

    # Get quiz set
    quiz_set = quiz_bank.get(predicted_topic, pd.DataFrame())

    return {
        "Next Topic": predicted_topic,
        "Message": message,
        "Quiz Set": quiz_set
    }


In [135]:
def recommend_next_quiz_set(row):
    import numpy as np
    import pandas as pd

    current_topic = row['topic'].strip().lower()
    current_diff = row['difficulty'].strip().lower()
    score = row['score']

    # Define difficulty-topic map
    difficulty_topic_map = {
        'easy': ['kinematics', 'optics'],
        'medium': ['mechanics', 'thermodynamics'],
        'hard': ['electricity', 'modern physics']
    }

    # Difficulty levels in order
    levels = ['easy', 'medium', 'hard']

    def get_difficulty_from_topic(topic):
        for level, topics in difficulty_topic_map.items():
            if topic in topics:
                return level
        return None

    # Decide next difficulty level based on score
    if score < 40:
        next_diff = levels[max(0, levels.index(current_diff) - 1)]
    elif 40 <= score <= 70:
        next_diff = current_diff
    elif score <= 90:
        next_diff = levels[min(len(levels) - 1, levels.index(current_diff) + 1)]
    else:
        predicted_topic = 'mixed questions'
        message = f"🎉 Excellent work! You're ready for a mixed practice quiz!"
        quiz_set = quiz_bank.get(predicted_topic, pd.DataFrame())
        return {
            "Next Topic": predicted_topic,
            "Message": message,
            "Quiz Set": quiz_set
        }

    # Get possible topics in the new difficulty level
    possible_topics = [
        topic for topic in difficulty_topic_map[next_diff]
        if topic != current_topic
    ]

    if possible_topics:
        predicted_topic = np.random.choice(possible_topics)
    else:
        predicted_topic = current_topic  # fallback

    # Feedback message
    if score < 40:
        message = f"😐 You need more practice. Let's try an easier topic: {predicted_topic.title()}."
    elif score <= 70:
        message = f"🙂 You're doing okay. Try another topic at the same level: {predicted_topic.title()}."
    else:
        message = f"🔥 Great job! Let's move to a harder topic: {predicted_topic.title()}."

    # Get quiz set
    quiz_set = quiz_bank.get(predicted_topic, pd.DataFrame())

    return {
        "Next Topic": predicted_topic,
        "Message": message,
        "Quiz Set": quiz_set
    }


In [138]:
student = df_filtered.iloc[0]
result = recommend_next_quiz_set(student)

print(f"Previous topic: {student['topic']}")
print(result['Message'])
print(f"\n📘 Recommended Quiz on: {result['Next Topic']}")


Previous topic: Thermodynamics
🙂 You're doing okay. Try another topic at the same level: Mechanics.

📘 Recommended Quiz on: mechanics


In [68]:
#evaluating the model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Ensure X_test and y_test from train_test_split are used
# These variables are defined in cell BOEUjYFgxgd-
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.5263157894736842
Confusion Matrix:
 [[13  0  3  6  0  0]
 [ 0 13  0  0  0  8]
 [ 5  0  8  4  0  0]
 [ 4  0  3  4  0  0]
 [ 0  0  0  0  3  0]
 [ 0 12  0  0  0  9]]
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.59      0.59        22
           1       0.52      0.62      0.57        21
           2       0.57      0.47      0.52        17
           4       0.29      0.36      0.32        11
           5       1.00      1.00      1.00         3
           6       0.53      0.43      0.47        21

    accuracy                           0.53        95
   macro avg       0.58      0.58      0.58        95
weighted avg       0.54      0.53      0.53        95



In [140]:
for i in range(10):
    student = df_filtered.iloc[i]
    result = recommend_next_quiz_set(student)
    print(f"\nStudent {student['student_id']}: {result['Message']}")
    print(f"Recommended Topic: {result['Next Topic']}")



Student 1099: 🙂 You're doing okay. Try another topic at the same level: Mechanics.
Recommended Topic: mechanics

Student 1014: 🙂 You're doing okay. Try another topic at the same level: Thermodynamics.
Recommended Topic: thermodynamics

Student 1065: 🙂 You're doing okay. Try another topic at the same level: Mechanics.
Recommended Topic: mechanics

Student 1100: 🙂 You're doing okay. Try another topic at the same level: Thermodynamics.
Recommended Topic: thermodynamics

Student 1008: 🔥 Great job! Let's move to a harder topic: Electricity.
Recommended Topic: electricity

Student 1060: 😐 You need more practice. Let's try an easier topic: Kinematics.
Recommended Topic: kinematics

Student 1080: 🙂 You're doing okay. Try another topic at the same level: Thermodynamics.
Recommended Topic: thermodynamics

Student 1018: 😐 You need more practice. Let's try an easier topic: Kinematics.
Recommended Topic: kinematics

Student 1044: 😐 You need more practice. Let's try an easier topic: Optics.
Recomme

In [141]:
import joblib

# Save the trained model to a file
joblib.dump(model, "quiz_recommender_model.pkl")
print("✅ Model saved as quiz_recommender_model.pkl")
from google.colab import files
files.download("quiz_recommender_model.pkl")


✅ Model saved as quiz_recommender_model.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [144]:
# For input topics
print("Topic Distribution:\n", df_filtered['topic'].value_counts())

# For target labels (if predicting next topic)
print("\nRecommended Next Topic Distribution:\n", df_filtered['recommended_next_topic'].value_counts())


Topic Distribution:
 topic
Mixed Questions     105
Kinematics          103
Mechanics            98
Optics               62
Electricity          50
Thermodynamics       35
Modern Physics       18
Name: count, dtype: int64

Recommended Next Topic Distribution:
 recommended_next_topic
mixed questions    120
kinematics          77
electricity         60
thermodynamics      58
mechanics           57
modern physics      54
optics              45
Name: count, dtype: int64


In [145]:
grouped = subject_df_clean.groupby('recommended_next_topic').agg({
    'score': ['mean', 'median'],
    'time_taken': ['mean', 'median'],
    'quiz_attempt': 'mean'
})
print(grouped)


                            score         time_taken        quiz_attempt
                             mean median        mean median         mean
recommended_next_topic                                                  
electricity             69.594872   69.0  157.943590  157.0     5.435897
kinematics              28.853081   29.0  164.289100  163.0     5.327014
mechanics               49.921053   50.0  160.663158  151.0     5.589474
mixed questions         90.094629   90.0  171.723785  178.0     5.265985
modern physics          69.423729   69.0  172.440678  167.0     5.779661
optics                  29.298780   29.0  167.152439  168.0     5.725610
thermodynamics          49.866279   50.5  160.593023  159.5     5.372093


In [151]:
from collections import Counter
from sklearn.utils import resample

class_counts = Counter(df_filtered['recommended_next_topic'])
print("Class counts:", class_counts)


Class counts: Counter({'mixed questions': 120, 'kinematics': 77, 'electricity': 60, 'thermodynamics ': 58, 'mechanics': 57, 'modern physics': 54, 'optics': 45})


In [152]:
import pandas as pd
import os

# Folder where you want to save the quizzes
output_folder = "topic_quizzes"
os.makedirs(output_folder, exist_ok=True)

# Group by topic and create topic-wise quizzes of 20–30 questions
topic_groups = df_filtered.groupby("topic")

for topic, group in topic_groups:
    quiz = group.sample(n=min(30, len(group)))  # take 30 or fewer questions
    filename = f"{topic.replace(' ', '_')}_quiz.csv"
    filepath = os.path.join(output_folder, filename)
    quiz.to_csv(filepath, index=False)
    print(f"✅ Saved quiz for topic: {topic} → {filepath}")


✅ Saved quiz for topic: Electricity → topic_quizzes/Electricity_quiz.csv
✅ Saved quiz for topic: Kinematics → topic_quizzes/Kinematics_quiz.csv
✅ Saved quiz for topic: Mechanics → topic_quizzes/Mechanics_quiz.csv
✅ Saved quiz for topic: Mixed Questions  → topic_quizzes/Mixed_Questions__quiz.csv
✅ Saved quiz for topic: Modern Physics → topic_quizzes/Modern_Physics_quiz.csv
✅ Saved quiz for topic: Optics → topic_quizzes/Optics_quiz.csv
✅ Saved quiz for topic: Thermodynamics → topic_quizzes/Thermodynamics_quiz.csv
